# Calculate Earth-Relative Atmospheric Angular Momentum

#### This notebook calculates earth-relative atmospheric angular momentum using data from the 20th Century Reanalysis Project Version 3 (https://psl.noaa.gov/data/20thC_Rean/). The zonal (u) wind data on constant pressure levels needed for this calculation is available at: https://psl.noaa.gov/thredds/catalog/Datasets/catalog.html.  

In [1]:
# Import the needed Python libraries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

#### Global earth-relative AAM ($M_R$) can be calculated using the equation below:

### $$ M_R = \frac{a^3}{g} \int_{-\pi/2}^{\pi/2} \int_{0}^{2\pi} \int_{p_0}^{p} \cos^{2} \phi d\phi d\lambda udp $$

#### where a is Earth’s radius in meters, g is the gravitational acceleration in $ms^{-2}$, $\phi$ is latitude in radians, $\lambda$ is longitude in radians, u is the zonal wind speed in $ms^{-1}$, and p is pressure in Pascals. Global relative AAM is calculated by integrating the zonal wind through the depth of the atmosphere over all latitudes and longitudes. The units of $M_R$ are $ kg \cdot m^{2} \cdot s^{-1}. $

In [ ]:
# Define the constants
gravity = 9.81 # gravitational acceleration; m/s^2
a = 6371220. # Earth's radius; m

# Convert the latitude and longitude deltas of 1.0 degrees (grid spacing of dataset) to radians
dlat, dlon = np.deg2rad(1.0), np.deg2rad(1.0)

# dp is the pressure difference in Pascals between subsequent 20CR vertical levels
# The first & last pressure differences are set to zero as they are located at the integral bounds
dps = [0.,500.,1000.,1000.,2000.,2000., 3000., 5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000., 2500.,2500.,2500.,2500., 0.]

#### The calc_AAM function calculates AAM at each latitude and longitude for one year.

In [ ]:
def calc_AAM(year):
    
    # Read in 1 year of 20CR u-wind data using Xarray, sort the pressure levels from low to high, change order of coordinates (makes dps array easier to create)
    ds = xr.open_dataset(f'/home/scratch/20CR_v3/uwnd.{year}.nc')
    ds = ds.sortby('level', ascending=True)
    ds = ds.uwnd.transpose("lat","lon","time","level")
    
    # Use np.tile to create a NumPy array of the above dps values with the same shape as the variable uwnd
    dprs = np.tile(dps, (181, 360, len(ds.time), 1)) 
    
    # Convert dprs from a numpy array to an Xarray DataArray
    dprs = xr.DataArray(dprs, dims=['lat', 'lon', 'time', 'level'])
    
    # Calculate udp (ds is the uwind speed; dprs are the pressure differences between levels)
    UDP = ds * dprs  #units= m/s * Pa = kg/s^3
    
    # Vertical integral of UDP
    vint_UDP = UDP.sum(dim='level') #units = kg/s^3
    
    # Calculate Earth-relative AAM at each latitude & longitude; units: kgs^-3 * m^3/ms^-2 = kgm^2s^-1 (angular momentum)
    Mr_by_latlon = vint_UDP * np.cos(np.deg2rad(ds.lat))**2 * dlat * dlon * a**3 / gravity
    ds.close()

    # Write the Xarray dataset Mr_by_latlon to a netcdf file
    Mr_by_latlon.to_netcdf(f'/home/scratch/20CR_v3/AAM_{year}.nc')
      
    del Mr
    del UDP
    del vint_UDP
    return year

#### Calculate AAM for the full 20th Century Reanalysis V3 dataset (1836-2015) using multiprocessing.

In [ ]:
years = np.arange(1836,2016,1)
#from multiprocessing import Pool
#pool = Pool(10)      # Create a multiprocessing Pool
#pool.map(calc_AAM, years)

#### Combine all the yearly AAM files into a single NetCDF.

In [ ]:
# Open AAM files for all years in the 20CR dataset, concatenate them, and rename the data variable
ds = xr.open_mfdataset('/home/scratch/20CR_v3/AAM_*.nc')
ds = ds.rename(name_dict={'__xarray_dataarray_variable__': 'Mr_by_latlon'})
ds

#### Calculate AAM by latitude and total/global AAM. 

In [ ]:
# Sum AAM across the longitude dimension to get AAM by latitude; add to the Xarray dataset
Mr_by_lat = ds.Mr_by_latlon.sum(dim='lon')
ds['Mr_by_lat'] = Mr_by_lat

# Calculate global/total AAM by summing AAM across all latitudes; add to the Xarray dataset
Mr = Mr_by_lat.sum(dim='lat')
ds['Mr'] = Mr
ds

#### Calculate the standardized anomalies of global AAM. 

In [ ]:
# Calculate daily mean Mr
Mr_daily_mean = ds.Mr.groupby('time.dayofyear').mean(dim='time')

# Calculate the daily standard deviation of Mr
Mr_daily_stdev = ds.Mr.groupby('time.dayofyear').std(dim='time')

In [ ]:
stdanom = []

# Calculate the standardized anomalies for the full dataset period; append them to a list
for i in ds.time.values:
    
    Mr_current = ds.Mr.sel(time=str(i))
    current_day = dt.datetime.strptime(str(Mr_current.time.values)[0:10], '%Y-%m-%d')
    dayofyear = int(current_day.strftime('%j'))
    
    # Select the daily mean and daily standard deviation of Mr
    Mr_daily_mn = Mr_daily_mean.sel(dayofyear = dayofyear)
    Mr_daily_std = Mr_daily_stdev.sel(dayofyear = dayofyear)
    
    # Calculate the standardized anomaly of Mr
    Mr_stdanom = (Mr_current.values - Mr_daily_mn.values) / Mr_daily_std.values

    stdanom.append(Mr_stdanom)

In [ ]:
# Add standardized anomalies to the Xarray dataset
ds['Mr_stdanom'] = ('time', stdanom)
ds

#### Create a NetCDF file containing Mr_by_latlon, Mr_by_lat, Mr, and the standardized anomalies of Mr for the full 20CR period.

In [ ]:
# Create a netcdf file from the Xarray dataset containing Mr_by_latlon, Mr_by_lat, Mr, & standardized anomalies of Mr
ds.to_netcdf('/home/scratch/20CR_v3/AAM_1836_2015.nc')